In [1]:
"""
=============================================================================
Sprint 2 - Data Cleaning Pipeline
PGA Outrights Betting Project (2019-2024)
=============================================================================
"""

import pandas as pd
import numpy as np
from pathlib import Path
import sys

# PATH CONFIGURATION
data_path = Path.cwd().parent / 'src' / 'Positive_EV_Repo' / 'data'
if str(data_path) not in sys.path:
    sys.path.append(str(data_path))
    
# IMPORT CUSTOM UTILITIES
from devig_utils import american_to_decimal, implied_from_american, proportional_devig

# DISPLAY SETTINGS
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', '{:.6f}'.format)

print("✓ Imports successful")
print(f"✓ Loaded devig_utils from: {data_path / 'devig_utils.py'}")

✓ Imports successful
✓ Loaded devig_utils from: c:\Users\mattr\OneDrive\Desktop\Positive_EV_Project\src\Positive_EV_Repo\data\devig_utils.py


In [3]:
# ============================================================================
# CELL 2: Load Raw Historical Outrights Data
# ============================================================================

print("="*70)
print("LOADING RAW DATA")
print("="*70)

# FILE LOCATION LOGIC
project_root = Path.cwd().parent

possible_paths = [
    project_root / 'data' / 'interim' / 'final_dataset.csv',
    project_root / 'src' / 'Positive_EV_Repo' / 'data' / 'interim' / 'hist_outrights.csv',
]

raw_path = None
for path in possible_paths:
    if path.exists():
        raw_path = path
        break

if raw_path is None:
    print("⚠️  File not found in expected locations. Searching project...")
    found = list(project_root.rglob('hist_outrights.csv'))
    
    if found:
        raw_path = found[0]
        print(f"⚠️  Found file at unexpected location: {raw_path}")
    else:
        raise FileNotFoundError(
            "hist_outrights.csv not found. Please run fetch_datagolf.py first:\n"
            "  cd src/Positive_EV_Repo/data\n"
            "  python fetch_datagolf.py"
        )

# LOAD DATA
df_raw = pd.read_csv(raw_path)

# INITIAL DATA INSPECTION
print(f"\n✓ Loaded from: {raw_path}")
print(f"Raw data shape: {df_raw.shape[0]:,} rows × {df_raw.shape[1]} columns")

print(f"\nColumns ({len(df_raw.columns)}):")
print(df_raw.columns.tolist())

print(f"\nData types:")
print(df_raw.dtypes)

print(f"\nFirst 3 rows (preview):")
display(df_raw.head(3))

LOADING RAW DATA

✓ Loaded from: c:\Users\mattr\OneDrive\Desktop\Positive_EV_Project\data\interim\final_dataset.csv
Raw data shape: 9,997 rows × 17 columns

Columns (17):
['bet_outcome_numeric', 'bet_outcome_text', 'close_odds', 'close_time', 'dg_id', 'open_odds', 'open_time', 'outcome', 'player_name', 'event_id', 'event_name', 'start_date', 'tour', 'book', 'market', 'close_time_dt', 'year']

Data types:
bet_outcome_numeric     int64
bet_outcome_text       object
close_odds              int64
close_time             object
dg_id                   int64
open_odds               int64
open_time              object
outcome                object
player_name            object
event_id                int64
event_name             object
start_date             object
tour                   object
book                   object
market                 object
close_time_dt          object
year                    int64
dtype: object

First 3 rows (preview):


,bet_outcome_numeric,bet_outcome_text,close_odds,close_time,dg_id,open_odds,open_time,outcome,player_name,event_id,event_name,start_date,tour,book,market,close_time_dt,year
0,0,loss,550,2025-01-02 11:38,19895,500,2024-12-30 14:35,T30,"Schauffele, Xander",16,The Sentry,2025-01-02,pga,draftkings,win,2025-01-02 11:38:00,2025
1,0,loss,900,2025-01-02 11:38,14139,1100,2024-12-30 14:35,T26,"Thomas, Justin",16,The Sentry,2025-01-02,pga,draftkings,win,2025-01-02 11:38:00,2025
2,0,loss,1100,2025-01-02 11:38,22085,1000,2024-12-30 14:35,2,"Morikawa, Collin",16,The Sentry,2025-01-02,pga,draftkings,win,2025-01-02 11:38:00,2025


In [4]:
# ============================================================================
# CELL 3: Initial Data Quality Assessment
# ============================================================================

print("="*70)
print("INITIAL DATA QUALITY ASSESSMENT")
print("="*70)

# 1. MARKET TYPES
print("\n1. Market Types:")
print(df_raw['market'].value_counts())
print(f"   → We will filter to 'win' market only (outrights)")

# 2. SPORTSBOOK COVERAGE
print("\n2. Sportsbook Coverage:")
print(df_raw['book'].value_counts())
print(f"   → Total books: {df_raw['book'].nunique()}")

# 3. TEMPORAL COVERAGE
print("\n3. Year Distribution:")
year_counts = df_raw['year'].value_counts().sort_index()
print(year_counts)
print(f"   → Coverage: {year_counts.index.min()} to {year_counts.index.max()}")

# 4. MISSING DATA ANALYSIS
print("\n4. Missing Data:")
missing = df_raw.isnull().sum()
missing_pct = 100 * missing / len(df_raw)

missing_df = pd.DataFrame({
    'Missing Count': missing,
    'Percentage': missing_pct
}).sort_values('Percentage', ascending=False)

missing_subset = missing_df[missing_df['Percentage'] > 0]

if len(missing_subset) > 0:
    print(missing_subset)
    print(f"\n   → {len(missing_subset)} columns have missing values")
else:
    print("  ✓ No missing values detected!")

# 5. EVENT IDENTIFICATION
print("\n5. Event Identification:")
if 'event_id' in df_raw.columns:
    print(f"  ✓ event_id column present")
    print(f"  Unique events: {df_raw['event_id'].nunique():,}")
    
    if df_raw['event_id'].isna().any():
        print(f"  ⚠️  {df_raw['event_id'].isna().sum()} rows have missing event_id")
else:
    print("  ⚠️  No event_id column found")
    print("     → Will construct from date/timestamp in next step")

print("\n" + "="*70)
print(f"✓ Initial assessment complete")
print(f"  Next: Filter to win market and parse outcomes")
print("="*70)

INITIAL DATA QUALITY ASSESSMENT

1. Market Types:
market
win    9997
Name: count, dtype: int64
   → We will filter to 'win' market only (outrights)

2. Sportsbook Coverage:
book
fanduel       5004
draftkings    4993
Name: count, dtype: int64
   → Total books: 2

3. Year Distribution:
year
2025    9997
Name: count, dtype: int64
   → Coverage: 2025 to 2025

4. Missing Data:
  ✓ No missing values detected!

5. Event Identification:
  ✓ event_id column present
  Unique events: 41

✓ Initial assessment complete
  Next: Filter to win market and parse outcomes


In [5]:
# ============================================================================
# CELL 4: Filter to Win Market Only
# ============================================================================

print("="*70)
print("STEP 1: FILTER TO WIN MARKET")
print("="*70)

# COUNT BEFORE FILTERING
initial_count = len(df_raw)
print(f"\nStarting dataset:")
print(f"  Total rows: {initial_count:,}")
print(f"  Market breakdown:")
print(df_raw['market'].value_counts())

# APPLY FILTER
df_win = df_raw[df_raw['market'] == 'win'].copy()

# COUNT AFTER FILTERING
dropped_count = initial_count - len(df_win)

print(f"\nAfter filtering to 'win' market:")
print(f"  Remaining rows: {len(df_win):,}")
print(f"  Dropped rows (other markets): {dropped_count:,}")
print(f"  Percentage retained: {100 * len(df_win) / initial_count:.1f}%")

# SANITY CHECK
assert df_win['market'].nunique() == 1, "ERROR: Multiple markets still present!"
assert df_win['market'].iloc[0] == 'win', "ERROR: Wrong market retained!"

print(f"\n✓ Filter successful - dataset contains only outright winner bets")

# Show sample of filtered data
print(f"\nSample of win market data (first 5 rows):")
display(df_win[['player_name', 'close_odds', 'outcome', 'year', 'book']].head())

STEP 1: FILTER TO WIN MARKET

Starting dataset:
  Total rows: 9,997
  Market breakdown:
market
win    9997
Name: count, dtype: int64

After filtering to 'win' market:
  Remaining rows: 9,997
  Dropped rows (other markets): 0
  Percentage retained: 100.0%

✓ Filter successful - dataset contains only outright winner bets

Sample of win market data (first 5 rows):


,player_name,close_odds,outcome,year,book
0,"Schauffele, Xander",550,T30,2025,draftkings
1,"Thomas, Justin",900,T26,2025,draftkings
2,"Morikawa, Collin",1100,2,2025,draftkings
3,"Aberg, Ludvig",1600,T5,2025,draftkings
4,"Cantlay, Patrick",1600,T15,2025,draftkings


In [6]:
# ============================================================================
# CELL 5: Parse Outcomes to Create Binary Target Variable (Y)
# ============================================================================

print("="*70)
print("STEP 2: PARSE OUTCOMES TO CREATE TARGET (Y)")
print("="*70)

# DEFINE PARSING FUNCTION
def parse_outcome_to_winner(outcome_str):
    """
    Convert DataGolf outcome string to binary winner flag.
    
    Logic:
        - Winner is indicated by exactly "1" (no tie) or "T1" (co-winner)
        - Everything else (2nd place, missed cut, etc.) is 0
        - Missing outcomes return NaN for later investigation
    """
    if pd.isna(outcome_str):
        return np.nan
    
    outcome_str = str(outcome_str).strip().upper()
    
    if outcome_str in ["1", "T1"]:
        return 1
    else:
        return 0

# APPLY PARSING TO ENTIRE DATASET
print("\nParsing outcomes...")
df_win['Y'] = df_win['outcome'].apply(parse_outcome_to_winner)

# TARGET VARIABLE SUMMARY
print(f"\n✓ Target variable (Y) created:")
print(f"\nValue counts:")
print(df_win['Y'].value_counts())

win_rate = df_win['Y'].mean()
print(f"\nOverall win rate: {win_rate:.6f} ({100*win_rate:.4f}%)")

# MISSING OUTCOME CHECK
missing_outcomes = df_win['Y'].isna().sum()

if missing_outcomes > 0:
    print(f"\n⚠️  WARNING: {missing_outcomes} rows have missing outcomes")
    print("   This could indicate:")
    print("   - Tournament was canceled/postponed")
    print("   - Data collection error")
    print("   - Event is ongoing (live odds)")
    
    print("\nSample rows with missing outcome:")
    missing_sample = df_win[df_win['Y'].isna()][['player_name', 'outcome', 'year']].head()
    display(missing_sample)
    
    print("\n   → These rows will be flagged for potential removal")
else:
    print("\n✓ No missing outcomes - all events have results")

# DISTRIBUTION BY YEAR
print("\nWinners per year:")
winners_by_year = df_win.groupby('year')['Y'].sum()
print(winners_by_year)

print(f"\n   → Typical PGA Tour season has ~45 events")
print(f"   → Our data has {winners_by_year.mean():.1f} winners per year (avg)")

# SAMPLE OUTCOMES
print("\nSample outcome parsing (first 10 rows):")
sample_outcomes = df_win[['player_name', 'outcome', 'Y']].head(10)
display(sample_outcomes)

STEP 2: PARSE OUTCOMES TO CREATE TARGET (Y)

Parsing outcomes...

✓ Target variable (Y) created:

Value counts:
Y
0    9915
1      82
Name: count, dtype: int64

Overall win rate: 0.008202 (0.8202%)

✓ No missing outcomes - all events have results

Winners per year:
year
2025    82
Name: Y, dtype: int64

   → Typical PGA Tour season has ~45 events
   → Our data has 82.0 winners per year (avg)

Sample outcome parsing (first 10 rows):


,player_name,outcome,Y
0,"Schauffele, Xander",T30,0
1,"Thomas, Justin",T26,0
2,"Morikawa, Collin",2,0
3,"Aberg, Ludvig",T5,0
4,"Cantlay, Patrick",T15,0
5,"Im, Sungjae",3,0
6,"Matsuyama, Hideki",1,1
7,"Burns, Sam",T8,0
8,"Theegala, Sahith",T36,0
9,"Hovland, Viktor",T36,0


In [7]:
# ============================================================================
# CELL 6: Create Event Identifier for Grouping
# ============================================================================

print("="*70)
print("STEP 3: CREATE EVENT IDENTIFIER")
print("="*70)

# PARSE TIMESTAMPS
print("\nParsing timestamp columns...")

df_win['close_time_dt'] = pd.to_datetime(df_win['close_time'], errors='coerce')
df_win['open_time_dt'] = pd.to_datetime(df_win['open_time'], errors='coerce')

bad_close = df_win['close_time_dt'].isna().sum()
bad_open = df_win['open_time_dt'].isna().sum()

if bad_close > 0:
    print(f"  ⚠️  {bad_close} rows have invalid close_time")
if bad_open > 0:
    print(f"  ⚠️  {bad_open} rows have invalid open_time")
    
if bad_close == 0 and bad_open == 0:
    print("  ✓ All timestamps parsed successfully")

# EXTRACT DATE (without time component)
df_win['event_date'] = df_win['close_time_dt'].dt.date

# CREATE OR VERIFY EVENT_ID
if 'event_id' not in df_win.columns:
    print("\nNo event_id column found. Constructing from year + close_time...")
    
    df_win['event_id'] = (
        df_win['year'].astype(str) + '_' + 
        df_win['event_date'].astype(str)
    )
    
    print(f"✓ Created event_id from year + close_time")
else:
    print(f"✓ event_id column already exists in data")

# EVENT SUMMARY STATISTICS
print(f"\nEvent summary:")
print(f"  Unique events: {df_win['event_id'].nunique():,}")
print(f"  Date range: {df_win['event_date'].min()} to {df_win['event_date'].max()}")

# FIELD SIZE DISTRIBUTION
event_sizes = df_win.groupby('event_id').size()

print(f"\nField sizes (players per event):")
print(event_sizes.describe())

print(f"\nSmallest events (top 5):")
print(event_sizes.nsmallest(5))

print(f"\nLargest events (top 5):")
print(event_sizes.nlargest(5))

small_events = (event_sizes < 100).sum()
large_events = (event_sizes > 160).sum()

if small_events > 0:
    print(f"\n  ⚠️  {small_events} events have < 100 players (invitational/limited field?)")
if large_events > 0:
    print(f"\n  ⚠️  {large_events} events have > 160 players (unusual)")

# SHOW SAMPLE EVENT
sample_event = event_sizes.index[0]
print(f"\nSample event: {sample_event}")
print(f"Field size: {event_sizes[sample_event]} players")

sample_event_data = df_win[df_win['event_id'] == sample_event][
    ['player_name', 'close_odds', 'outcome', 'Y']
].head(10)

print(f"\nTop 10 players (by odds) in this event:")
display(sample_event_data)

STEP 3: CREATE EVENT IDENTIFIER

Parsing timestamp columns...
  ✓ All timestamps parsed successfully
✓ event_id column already exists in data

Event summary:
  Unique events: 41
  Date range: 2025-01-02 to 2025-10-08

Field sizes (players per event):
count    41.000000
mean    243.829268
std      77.513838
min      60.000000
25%     156.000000
50%     283.000000
75%     309.000000
max     312.000000
dtype: float64

Smallest events (top 5):
event_id
60     60
28     98
16    118
27    138
34    140
dtype: int64

Largest events (top 5):
event_id
13     312
33     312
472    312
524    312
2      311
dtype: int64

  ⚠️  2 events have < 100 players (invitational/limited field?)

  ⚠️  29 events have > 160 players (unusual)

Sample event: 2
Field size: 311 players

Top 10 players (by odds) in this event:


,player_name,close_odds,outcome,Y
401,"Thomas, Justin",1200,2,0
402,"Im, Sungjae",1400,CUT,0
403,"Burns, Sam",1600,T29,0
404,"Cantlay, Patrick",1800,T5,0
405,"Finau, Tony",2200,CUT,0
406,"Kim, Tom",2800,CUT,0
407,"Greyserman, Max",3000,T7,0
408,"Kitayama, Kurt",3000,T58,0
409,"Clark, Wyndham",3000,CUT,0
410,"Thompson, Davis",3500,T51,0


In [11]:
# ============================================================================
# CELL 6: Identify Winners (Create Y Variable) - FIXED
# ============================================================================

print("\n" + "="*70)
print("STEP 5: IDENTIFY WINNERS (CREATE Y VARIABLE)")
print("="*70)

# The outcome column shows finishing position
# outcome='1' means 1st place (winner)
# outcome='T14' means tied 14th place (loser)
df_win['Y'] = (df_win['outcome'].astype(str) == '1').astype(int)

print(f"\nWinner identification:")
print(f"  Total rows: {len(df_win):,}")
print(f"  Winners (Y=1): {df_win['Y'].sum():,}")
print(f"  Losers (Y=0): {(df_win['Y']==0).sum():,}")
print(f"  Win rate: {100*df_win['Y'].mean():.2f}%")

# Show examples
print(f"\nSample winners:")
print(df_win[df_win['Y']==1][['player_name', 'outcome', 'bet_outcome_text']].head(3))

print(f"\nSample losers:")
print(df_win[df_win['Y']==0][['player_name', 'outcome', 'bet_outcome_text']].head(3))

# Validate: should be ~1-2% winners (1 winner per 50-150 players)
expected_win_rate = 1 / df_win.groupby('dg_id').size().mean()
print(f"\nExpected win rate: {100*expected_win_rate:.2f}%")

if df_win['Y'].mean() > 0.1:  # More than 10% winners is suspicious
    print("\n⚠️ WARNING: Win rate seems too high!")
    print("Check winner identification logic")
"""

**Expected Output After Fix:**
```
Winners (Y=1): 2
Losers (Y=0): 622
Win rate: 0.32%
Events with exactly 1 winner: 2 / 2
Clean event coverage: 100.0%
"""



STEP 5: IDENTIFY WINNERS (CREATE Y VARIABLE)

Winner identification:
  Total rows: 9,997
  Winners (Y=1): 82
  Losers (Y=0): 9,915
  Win rate: 0.82%

Sample winners:
           player_name outcome bet_outcome_text
6    Matsuyama, Hideki       1     paid in full
65   Matsuyama, Hideki       1     paid in full
158       Taylor, Nick       1     paid in full

Sample losers:
          player_name outcome bet_outcome_text
0  Schauffele, Xander     T30             loss
1      Thomas, Justin     T26             loss
2    Morikawa, Collin       2             loss

Expected win rate: 6.66%


'\n\n**Expected Output After Fix:**\n```\nWinners (Y=1): 2\nLosers (Y=0): 622\nWin rate: 0.32%\nEvents with exactly 1 winner: 2 / 2\nClean event coverage: 100.0%\n'

In [12]:
# ============================================================================
# CELL 7: Validate Winner Counts per Event
# ============================================================================

print("="*70)
print("STEP 4: VALIDATE WINNER COUNTS PER EVENT")
print("="*70)

# COUNT WINNERS PER EVENT
winner_counts = df_win.groupby('event_id')['Y'].sum()

# SUMMARY STATISTICS
total_events = len(winner_counts)
good_events = (winner_counts == 1).sum()
zero_winner_events = (winner_counts == 0).sum()
multi_winner_events = (winner_counts > 1).sum()

print(f"\nWinner count validation:")
print(f"  Total events: {total_events:,}")
print(f"  Events with exactly 1 winner: {good_events:,} ({100*good_events/total_events:.1f}%)")
print(f"  Events with 0 winners: {zero_winner_events:,}")
print(f"  Events with 2+ winners: {multi_winner_events:,}")

# INVESTIGATE ZERO-WINNER EVENTS
if zero_winner_events > 0:
    print(f"\n{'='*70}")
    print(f"⚠️  ISSUE: {zero_winner_events} events have NO winner")
    print(f"{'='*70}")
    
    no_winner = winner_counts[winner_counts == 0]
    
    print(f"\nEvents with 0 winners:")
    print(no_winner.head(10))
    
    sample_event = no_winner.index[0]
    print(f"\nSample data from event: {sample_event}")
    sample_data = df_win[df_win['event_id'] == sample_event][
        ['event_id', 'player_name', 'outcome', 'Y', 'year']
    ].head()
    display(sample_data)
    
    print("\nPossible causes:")
    print("  - Tournament canceled/postponed")
    print("  - Incomplete data (still in progress)")
    print("  - Outcome parsing failed")
    print("  → Recommend: Exclude these events from modeling")

# INVESTIGATE MULTI-WINNER EVENTS
if multi_winner_events > 0:
    print(f"\n{'='*70}")
    print(f"⚠️  ISSUE: {multi_winner_events} events have MULTIPLE winners")
    print(f"{'='*70}")
    
    multi_winner = winner_counts[winner_counts > 1]
    
    print(f"\nEvents with 2+ winners:")
    print(multi_winner.head(10))
    
    sample_event = multi_winner.index[0]
    print(f"\nSample data from event: {sample_event}")
    sample_data = df_win[df_win['event_id'] == sample_event][
        ['event_id', 'player_name', 'outcome', 'Y', 'book', 'close_time']
    ]
    
    winners_only = sample_data[sample_data['Y'] == 1]
    display(winners_only)
    
    print("\nPossible causes:")
    print("  - True co-winners (tied 1st, no playoff) - RARE but valid")
    print("  - Duplicate rows not yet removed (same player, multiple books/times)")
    print("  → Will be resolved in deduplication step (Cell 12)")

# FLAG PROBLEMATIC EVENTS
problematic_event_ids = winner_counts[winner_counts != 1].index

df_win['flag_bad_winner_count'] = df_win['event_id'].isin(problematic_event_ids)

flagged_rows = df_win['flag_bad_winner_count'].sum()
flagged_events = (winner_counts != 1).sum()

print(f"\n{'='*70}")
print(f"✓ Flagging complete:")
print(f"  Flagged rows: {flagged_rows:,} (in {flagged_events} events)")
print(f"  Clean rows: {len(df_win) - flagged_rows:,} (in {total_events - flagged_events} events)")
print(f"{'='*70}")

if flagged_rows > 0:
    print(f"\n⚠️  Note: Flagged events retained for now, will address in:")
    print(f"     - Deduplication (Cell 12)")
    print(f"     - Final validation (Cell 13)")
else:
    print(f"\n✓ All events have exactly 1 winner - data quality excellent!")

STEP 4: VALIDATE WINNER COUNTS PER EVENT

Winner count validation:
  Total events: 41
  Events with exactly 1 winner: 0 (0.0%)
  Events with 0 winners: 0
  Events with 2+ winners: 41

⚠️  ISSUE: 41 events have MULTIPLE winners

Events with 2+ winners:
event_id
2     2
3     2
4     2
5     2
6     2
7     2
9     2
10    2
11    2
12    2
Name: Y, dtype: int64

Sample data from event: 2


,event_id,player_name,outcome,Y,book,close_time
418,2,"Straka, Sepp",1,1,draftkings,2025-01-16 09:04
574,2,"Straka, Sepp",1,1,fanduel,2025-01-16 09:04



Possible causes:
  - True co-winners (tied 1st, no playoff) - RARE but valid
  - Duplicate rows not yet removed (same player, multiple books/times)
  → Will be resolved in deduplication step (Cell 12)

✓ Flagging complete:
  Flagged rows: 9,997 (in 41 events)
  Clean rows: 0 (in 0 events)

⚠️  Note: Flagged events retained for now, will address in:
     - Deduplication (Cell 12)
     - Final validation (Cell 13)


In [15]:
df_win.head()

,bet_outcome_numeric,bet_outcome_text,close_odds,close_time,dg_id,open_odds,open_time,outcome,player_name,event_id,event_name,start_date,tour,book,market,close_time_dt,year,Y,open_time_dt,event_date,flag_bad_winner_count
0,0,loss,550,2025-01-02 11:38,19895,500,2024-12-30 14:35,T30,"Schauffele, Xander",16,The Sentry,2025-01-02,pga,draftkings,win,2025-01-02 11:38:00,2025,0,2024-12-30 14:35:00,2025-01-02,True
1,0,loss,900,2025-01-02 11:38,14139,1100,2024-12-30 14:35,T26,"Thomas, Justin",16,The Sentry,2025-01-02,pga,draftkings,win,2025-01-02 11:38:00,2025,0,2024-12-30 14:35:00,2025-01-02,True
2,0,loss,1100,2025-01-02 11:38,22085,1000,2024-12-30 14:35,2,"Morikawa, Collin",16,The Sentry,2025-01-02,pga,draftkings,win,2025-01-02 11:38:00,2025,0,2024-12-30 14:35:00,2025-01-02,True
3,0,loss,1600,2025-01-02 11:38,23950,1800,2024-12-30 14:35,T5,"Aberg, Ludvig",16,The Sentry,2025-01-02,pga,draftkings,win,2025-01-02 11:38:00,2025,0,2024-12-30 14:35:00,2025-01-02,True
4,0,loss,1600,2025-01-02 11:38,15466,1800,2024-12-30 14:35,T15,"Cantlay, Patrick",16,The Sentry,2025-01-02,pga,draftkings,win,2025-01-02 11:38:00,2025,0,2024-12-30 14:35:00,2025-01-02,True


In [16]:
# ============================================================================
# CELL 8: Create df_win - Winners Only with Clean Odds
# ============================================================================

print("="*70)
print("STEP 5: CREATE ANALYSIS DATASET (WINNERS ONLY)")
print("="*70)

# START FROM df_raw
print(f"\nStarting with df_raw: {len(df_raw):,} rows")

# FILTER FOR WINNERS ONLY
df_win = df_raw[df_raw['bet_outcome_numeric'] == 1.0].copy()
print(f"After filtering for winners: {len(df_win):,} rows")

# CREATE REQUIRED COLUMNS
# Y column: Binary outcome (all 1s since we only have winners)
df_win['Y'] = 1

# event_id: Group bets by tournament (using year + close_time date)
df_win['event_id'] = df_win['year'].astype(str) + '_' + df_win['close_time'].str[:10]

print(f"\nCreated columns:")
print(f"  Y: Binary outcome (all 1s for winners)")
print(f"  event_id: {df_win['event_id'].nunique()} unique tournaments")

# CHECK FOR NULL VALUES
print(f"\n{'='*70}")
print("Checking for missing odds...")
null_close = df_win['close_odds'].isna().sum()
null_open = df_win['open_odds'].isna().sum()

print(f"  close_odds null: {null_close:,}")
print(f"  open_odds null: {null_open:,}")

# VALIDATE AMERICAN ODDS FORMAT
print("\nValidating American odds format...")

def is_valid_american_odds(odds_val):
    """Check if value is valid American odds."""
    if pd.isna(odds_val):
        return False
    
    # Handle numeric values (already converted)
    if isinstance(odds_val, (int, float)):
        return True
    
    # Handle string values
    odds_str = str(odds_val).strip()
    
    # Check format: should start with + or - OR be a plain number
    if odds_str.lstrip('+-').isdigit():
        return True
    
    return False

df_win['valid_odds'] = df_win['close_odds'].apply(is_valid_american_odds)

invalid_count = (~df_win['valid_odds']).sum()

print(f"\n  Valid odds: {df_win['valid_odds'].sum():,}")
print(f"  Invalid odds: {invalid_count:,}")

# SHOW EXAMPLES OF INVALID ODDS
if invalid_count > 0:
    print(f"\n⚠️  WARNING: {invalid_count} rows have invalid odds format")
    print("\nSample invalid odds (first 10):")
    invalid_sample = df_win[~df_win['valid_odds']][
        ['player_name', 'close_odds', 'year', 'event_id']
    ].head(10)
    print(invalid_sample.to_string(index=False))

# DROP INVALID ODDS
before_count = len(df_win)

df_win = df_win[df_win['valid_odds']].copy()
df_win.drop('valid_odds', axis=1, inplace=True)

after_count = len(df_win)
dropped = before_count - after_count

# SUMMARY
print(f"\n{'='*70}")
print(f"✓ df_win creation complete:")
print(f"  Winners from df_raw: {len(df_raw[df_raw['bet_outcome_numeric'] == 1.0]):,}")
print(f"  After odds validation: {after_count:,}")
print(f"  Dropped (invalid odds): {dropped:,}")
print(f"  Final df_win size: {after_count:,} winning bets")
print(f"{'='*70}")

print(f"\ndf_win preview:")
print(df_win[['player_name', 'close_odds', 'outcome', 'Y', 'event_id']].head())

print(f"\n✓ Ready for Cell 9 (odds conversion)!")

STEP 5: CREATE ANALYSIS DATASET (WINNERS ONLY)

Starting with df_raw: 9,997 rows
After filtering for winners: 82 rows

Created columns:
  Y: Binary outcome (all 1s for winners)
  event_id: 37 unique tournaments

Checking for missing odds...
  close_odds null: 0
  open_odds null: 0

Validating American odds format...

  Valid odds: 82
  Invalid odds: 0

✓ df_win creation complete:
  Winners from df_raw: 82
  After odds validation: 82
  Dropped (invalid odds): 0
  Final df_win size: 82 winning bets

df_win preview:
           player_name  close_odds outcome  Y         event_id
6    Matsuyama, Hideki        2000       1  1  2025_2025-01-02
65   Matsuyama, Hideki        2000       1  1  2025_2025-01-02
158       Taylor, Nick       10000       1  1  2025_2025-01-09
301       Taylor, Nick       11000       1  1  2025_2025-01-09
418       Straka, Sepp        5500       1  1  2025_2025-01-16

✓ Ready for Cell 9 (odds conversion)!


In [17]:
# ============================================================================
# CELL 9: Convert Odds to Decimal & Calculate Implied Probabilities
# ============================================================================

print("="*70)
print("STEP 6: CONVERT ODDS & CALCULATE IMPLIED PROBABILITIES")
print("="*70)

# ============================================================================
# CONVERT TO NUMERIC (ROBUST VERSION)
# ============================================================================
print("\nConverting American odds to numeric...")

# Handle both string and numeric types
def convert_american_odds(value):
    """Convert American odds (string or numeric) to integer"""
    if pd.isna(value):
        return None
    if isinstance(value, (int, float)):
        return int(value)
    # If string, remove + and spaces
    return int(str(value).replace('+', '').replace(' ', ''))

df_win['american_odds'] = df_win['close_odds'].apply(convert_american_odds)

print(f"  ✓ Converted {len(df_win):,} odds to numeric format")

# ============================================================================
# CONVERT TO DECIMAL ODDS
# ============================================================================
print("\nConverting to decimal odds...")

df_win['dec_odds'] = american_to_decimal(df_win['american_odds'])

print(f"  ✓ Calculated decimal odds")

# ============================================================================
# CALCULATE IMPLIED PROBABILITIES
# ============================================================================
print("\nCalculating raw implied probabilities...")

df_win['implied_raw'] = implied_from_american(df_win['american_odds'])

print(f"  ✓ Calculated implied probabilities")

# ============================================================================
# SHOW CONVERSION EXAMPLES
# ============================================================================
print("\nSample conversions (first 10 rows):")
conversion_cols = ['player_name', 'close_odds', 'american_odds', 'dec_odds', 'implied_raw']
conversion_sample = df_win[conversion_cols].head(10)

print(conversion_sample.to_string(index=False))

# ============================================================================
# VALIDATION CHECKS
# ============================================================================
print(f"\n{'='*70}")
print("Validation checks:")
print(f"{'='*70}")

# Check 1: All decimal odds should be >= 1.0
all_decimal_valid = (df_win['dec_odds'] >= 1.0).all()
print(f"  1. All decimal odds >= 1.0: {all_decimal_valid}")

if not all_decimal_valid:
    bad_decimal = df_win[df_win['dec_odds'] < 1.0]
    print(f"\n     ⚠️  ERROR: {len(bad_decimal)} rows have decimal odds < 1.0")
    print(bad_decimal[['player_name', 'close_odds', 'american_odds', 'dec_odds']].head())

# Check 2: All implied probabilities should be between 0 and 1
all_implied_valid = ((df_win['implied_raw'] > 0) & (df_win['implied_raw'] < 1)).all()
print(f"  2. All implied probs in (0, 1): {all_implied_valid}")

if not all_implied_valid:
    bad_implied = df_win[~((df_win['implied_raw'] > 0) & (df_win['implied_raw'] < 1))]
    print(f"\n     ⚠️  ERROR: {len(bad_implied)} rows have implied prob out of bounds")
    print(bad_implied[['player_name', 'close_odds', 'implied_raw']].head())

# ============================================================================
# DISTRIBUTION SUMMARY
# ============================================================================
print(f"\n✓ Odds distribution summary:")
print(df_win[['dec_odds', 'implied_raw']].describe())

print(f"\nInterpretation:")
median_dec = df_win['dec_odds'].median()
median_implied = df_win['implied_raw'].median()
print(f"  Median decimal odds: {median_dec:.2f} ({median_implied:.4f} implied)")
print(f"  → Typical player has ~{100*median_implied:.2f}% win probability (before de-vig)")

print(f"\n  Min odds: {df_win['dec_odds'].min():.2f} (favorite)")
print(f"  Max odds: {df_win['dec_odds'].max():.2f} (extreme longshot)")

STEP 6: CONVERT ODDS & CALCULATE IMPLIED PROBABILITIES

Converting American odds to numeric...
  ✓ Converted 82 odds to numeric format

Converting to decimal odds...
  ✓ Calculated decimal odds

Calculating raw implied probabilities...
  ✓ Calculated implied probabilities

Sample conversions (first 10 rows):
      player_name  close_odds  american_odds   dec_odds  implied_raw
Matsuyama, Hideki        2000           2000  21.000000     0.047619
Matsuyama, Hideki        2000           2000  21.000000     0.047619
     Taylor, Nick       10000          10000 101.000000     0.009901
     Taylor, Nick       11000          11000 111.000000     0.009009
     Straka, Sepp        5500           5500  56.000000     0.017857
     Straka, Sepp        6000           6000  61.000000     0.016393
  English, Harris        8000           8000  81.000000     0.012346
  English, Harris        9000           9000  91.000000     0.010989
    McIlroy, Rory        1200           1200  13.000000     0.076923


In [18]:
# ============================================================================
# CELL 10: Remove Bookmaker Vig (Proportional De-vig Method) - FIXED
# ============================================================================

print("="*70)
print("STEP 7: DE-VIG ODDS (PROPORTIONAL METHOD)")
print("="*70)

# PRE-FLIGHT CHECK
print(f"\nPre-flight check:")
print(f"  Rows: {len(df_win):,}")
print(f"  Events: {df_win['event_id'].nunique():,}")
print(f"  Has implied_raw: {'implied_raw' in df_win.columns}")

if len(df_win) == 0:
    raise ValueError("ERROR: df_win is empty!")

# DE-VIG USING TRANSFORM (SAFE METHOD)
print("\nCalculating overround per event...")

# Use transform to broadcast sum to all rows in each group
df_win['overround'] = df_win.groupby('event_id')['implied_raw'].transform('sum')

print(f"  ✓ Overround calculated")

# De-vig: divide each implied_raw by the event's overround
print("\nDe-vigging probabilities...")

df_win['p_book'] = df_win['implied_raw'] / df_win['overround']

print(f"  ✓ De-vig complete for {len(df_win):,} rows")

# VALIDATE DE-VIG RESULTS
print(f"\n{'='*70}")
print("Validation: De-vigged probabilities")
print(f"{'='*70}")

event_prob_sums = df_win.groupby('event_id')['p_book'].sum()

print(f"\nDe-vigged probability sums per event:")
print(f"  Count:  {len(event_prob_sums):,}")
print(f"  Mean:   {event_prob_sums.mean():.15f}")
print(f"  Std:    {event_prob_sums.std():.15e}")
print(f"  Min:    {event_prob_sums.min():.15f}")
print(f"  Max:    {event_prob_sums.max():.15f}")

all_sum_to_one = np.allclose(event_prob_sums, 1.0, atol=1e-10)
print(f"\n  All events sum to 1.0 (±1e-10): {all_sum_to_one}")

if not all_sum_to_one:
    print("\n  ⚠️  WARNING: Some events don't sum to exactly 1.0")
    bad_events = event_prob_sums[~np.isclose(event_prob_sums, 1.0, atol=1e-10)]
    print(f"  Problematic events: {len(bad_events)}")
    print(bad_events.head())

# OVERROUND (VIG) ANALYSIS
print(f"\n{'='*70}")
print("Bookmaker Margin (Overround) Analysis")
print(f"{'='*70}")

overround_stats = df_win.groupby('event_id')['overround'].first()

print(f"\nOverround statistics:")
print(overround_stats.describe())

print(f"\nInterpretation (as margin %):")
print(f"  Min:        {100*(overround_stats.min()-1):.2f}%")
print(f"  25th pct:   {100*(overround_stats.quantile(0.25)-1):.2f}%")
print(f"  Median:     {100*(overround_stats.median()-1):.2f}%")
print(f"  75th pct:   {100*(overround_stats.quantile(0.75)-1):.2f}%")
print(f"  Max:        {100*(overround_stats.max()-1):.2f}%")

print(f"\nTypical PGA outrights vig: 15-20%")
print(f"Our data median: {100*(overround_stats.median()-1):.1f}%")

# SHOW BEFORE/AFTER EXAMPLE
sample_event = df_win['event_id'].iloc[0]

print(f"\n{'='*70}")
print(f"Example: Event {sample_event}")
print(f"{'='*70}")

sample_df = df_win[df_win['event_id'] == sample_event][
    ['player_name', 'dec_odds', 'implied_raw', 'p_book', 'Y']
].head(10)

print(sample_df.to_string(index=False))

sample_implied_sum = df_win[df_win['event_id'] == sample_event]['implied_raw'].sum()
sample_pbook_sum = df_win[df_win['event_id'] == sample_event]['p_book'].sum()

print(f"\nBefore de-vig (raw implied):  Sum = {sample_implied_sum:.6f}")
print(f"After de-vig (p_book):        Sum = {sample_pbook_sum:.15f}")
print(f"Removed vig:                  {100*(sample_implied_sum-1):.2f}%")

print(f"\n✓ De-vig successful - probabilities now sum to 1.0 per event")

STEP 7: DE-VIG ODDS (PROPORTIONAL METHOD)

Pre-flight check:
  Rows: 82
  Events: 37
  Has implied_raw: True

Calculating overround per event...
  ✓ Overround calculated

De-vigging probabilities...
  ✓ De-vig complete for 82 rows

Validation: De-vigged probabilities

De-vigged probability sums per event:
  Count:  37
  Mean:   1.000000000000000
  Std:    1.850371707708594e-17
  Min:    1.000000000000000
  Max:    1.000000000000000

  All events sum to 1.0 (±1e-10): True

Bookmaker Margin (Overround) Analysis

Overround statistics:
count   37.000000
mean     0.138145
std      0.184567
min      0.005816
25%      0.024147
50%      0.048780
75%      0.153846
max      0.645161
Name: overround, dtype: float64

Interpretation (as margin %):
  Min:        -99.42%
  25th pct:   -97.59%
  Median:     -95.12%
  75th pct:   -84.62%
  Max:        -35.48%

Typical PGA outrights vig: 15-20%
Our data median: -95.1%

Example: Event 2025_2025-01-02
      player_name  dec_odds  implied_raw   p_book  Y
M

In [20]:
# ============================================================================
# CELL 11: Engineer Additional Features for Modeling - FIXED
# ============================================================================

print("="*70)
print("STEP 8: FEATURE ENGINEERING")
print("="*70)

# 1. FIELD SIZE
print("\n1. Creating field_size...")
df_win['field_size'] = df_win.groupby('event_id')['dg_id'].transform('count')
print(f"   ✓ field_size: Range {df_win['field_size'].min()}-{df_win['field_size'].max()}")

# 2. PRICE RANK
print("\n2. Creating price_rank...")
df_win['price_rank'] = df_win.groupby('event_id')['dec_odds'].rank(method='min')
print(f"   ✓ price_rank: 1 (favorite) to {int(df_win['price_rank'].max())} (longshot)")

min_ranks = df_win.groupby('event_id')['price_rank'].min()
assert (min_ranks == 1).all(), "ERROR: Some events missing rank-1 player!"

# 3. LOG ODDS
print("\n3. Creating log_odds...")
df_win['log_odds'] = np.log(df_win['dec_odds'])
print(f"   ✓ log_odds: Range {df_win['log_odds'].min():.2f} to {df_win['log_odds'].max():.2f}")

# 4. FAVORITE FLAGS
print("\n4. Creating favorite flags...")
df_win['is_top5_favorite'] = (df_win['price_rank'] <= 5).astype(bool)
df_win['is_top10_favorite'] = (df_win['price_rank'] <= 10).astype(bool)
print(f"   ✓ is_top5_favorite:  {df_win['is_top5_favorite'].sum():,} rows ({100*df_win['is_top5_favorite'].mean():.1f}%)")
print(f"   ✓ is_top10_favorite: {df_win['is_top10_favorite'].sum():,} rows ({100*df_win['is_top10_favorite'].mean():.1f}%)")

# 5. TEMPORAL FEATURES - FIXED
print("\n5. Creating temporal features...")

# Force re-parse of close_time to datetime
df_win['close_time_dt'] = pd.to_datetime(df_win['close_time'], errors='coerce')

# Check for parsing failures
null_timestamps = df_win['close_time_dt'].isna().sum()
if null_timestamps > 0:
    print(f"   ⚠️  {null_timestamps} timestamps could not be parsed")

# Extract temporal features
df_win['year_corrected'] = df_win['close_time_dt'].dt.year
df_win['month'] = df_win['close_time_dt'].dt.month
df_win['day_of_week'] = df_win['close_time_dt'].dt.dayofweek

# Handle any NaN values from failed parsing
if df_win['year_corrected'].isna().any():
    print(f"   ⚠️  Filling {df_win['year_corrected'].isna().sum()} missing years with modal year")
    df_win['year_corrected'] = df_win['year_corrected'].fillna(df_win['year_corrected'].mode()[0])

print(f"   ✓ year_corrected: {int(df_win['year_corrected'].min())} to {int(df_win['year_corrected'].max())}")
print(f"   ✓ month: {int(df_win['month'].min())} to {int(df_win['month'].max())}")
print(f"   ✓ day_of_week: {int(df_win['day_of_week'].min())} to {int(df_win['day_of_week'].max())}")

# 6. BOOK ENCODING
print("\n6. Creating book encoding...")
df_win['book_encoded'] = df_win['book'].astype('category').cat.codes
print(f"   ✓ book_encoded: {df_win['book'].nunique()} unique books")
print(f"     Books: {df_win['book'].unique()}")

# SUMMARY
print(f"\n{'='*70}")
print("✓ Feature engineering complete")
print(f"{'='*70}")

print("\nNew features added:")
print("  - field_size, price_rank, log_odds")
print("  - is_top5_favorite, is_top10_favorite")
print("  - year_corrected, month, day_of_week")
print("  - book_encoded")

print(f"\nFeature summary:")
feature_cols = ['field_size', 'price_rank', 'log_odds', 'overround']
available_cols = [col for col in feature_cols if col in df_win.columns]
if available_cols:
    print(df_win[available_cols].describe())

print(f"\nSample (first 10 rows):")
display_cols = ['player_name', 'dec_odds', 'price_rank', 'p_book', 'field_size', 'is_top5_favorite', 'Y']
available = [col for col in display_cols if col in df_win.columns]
if available:
    print(df_win[available].head(10).to_string(index=False))

STEP 8: FEATURE ENGINEERING

1. Creating field_size...
   ✓ field_size: Range 2-4

2. Creating price_rank...
   ✓ price_rank: 1 (favorite) to 4 (longshot)

3. Creating log_odds...
   ✓ log_odds: Range 1.13 to 5.99

4. Creating favorite flags...
   ✓ is_top5_favorite:  82 rows (100.0%)
   ✓ is_top10_favorite: 82 rows (100.0%)

5. Creating temporal features...
   ✓ year_corrected: 2025 to 2025
   ✓ month: 1 to 10
   ✓ day_of_week: 2 to 3

6. Creating book encoding...
   ✓ book_encoded: 2 unique books
     Books: ['draftkings' 'fanduel']

✓ Feature engineering complete

New features added:
  - field_size, price_rank, log_odds
  - is_top5_favorite, is_top10_favorite
  - year_corrected, month, day_of_week
  - book_encoded

Feature summary:
       field_size  price_rank  log_odds  overround
count   82.000000   82.000000 82.000000  82.000000
mean     2.390244    1.414634  3.535011   0.142072
std      0.797467    0.769018  1.195737   0.181543
min      2.000000    1.000000  1.131402   0.005816


In [21]:
# ============================================================================
# CELL 12: Remove Duplicate Rows
# ============================================================================

print("="*70)
print("STEP 9: REMOVE DUPLICATES")
print("="*70)

# 1. CHECK FOR EXACT DUPLICATES
print("\n1. Checking for exact duplicate rows...")

exact_dupes = df_win.duplicated().sum()

print(f"   Exact duplicate rows: {exact_dupes:,}")

if exact_dupes > 0:
    print(f"   → Dropping {exact_dupes} exact duplicates...")
    df_win = df_win.drop_duplicates()
    print(f"   ✓ Done. Remaining rows: {len(df_win):,}")
else:
    print(f"   ✓ No exact duplicates found")

# 2. CHECK FOR (EVENT, PLAYER) DUPLICATES
print("\n2. Checking for (event_id, player) duplicates...")

key_dupes = df_win.duplicated(subset=['event_id', 'dg_id']).sum()

print(f"   Duplicate (event_id, dg_id) pairs: {key_dupes:,}")

if key_dupes > 0:
    print(f"\n   ⚠️  WARNING: Same player appears multiple times in same event")
    print(f"   This is likely due to:")
    print(f"      - Multiple sportsbooks (DraftKings + FanDuel)")
    print(f"      - Multiple timestamps (opening + closing lines)")
    print(f"      - Data collection running at different times")
    
    # SHOW EXAMPLE DUPLICATE
    dupe_mask = df_win.duplicated(subset=['event_id', 'dg_id'], keep=False)
    dupe_pairs = df_win[dupe_mask]
    
    if len(dupe_pairs) > 0:
        sample_player = dupe_pairs.iloc[0]
        sample_event = sample_player['event_id']
        sample_dg_id = sample_player['dg_id']
        
        print(f"\n   Example: {sample_player['player_name']} in {sample_event}")
        print(f"   This player appears multiple times with different odds:\n")
        
        example_dupes = dupe_pairs[
            (dupe_pairs['event_id'] == sample_event) & 
            (dupe_pairs['dg_id'] == sample_dg_id)
        ][['player_name', 'book', 'close_time', 'close_odds', 'dec_odds', 'Y']]
        
        display(example_dupes)
    
    # DEDUPLICATION STRATEGY
    print(f"\n{'='*70}")
    print("   DEDUPLICATION STRATEGY:")
    print("   Keep row with LATEST close_time per (event_id, dg_id)")
    print("   Rationale: Closing lines are most accurate, reflect latest info")
    print(f"{'='*70}")
    
    before_dedup = len(df_win)
    
    # Sort by timestamp
    df_win = df_win.sort_values('close_time_dt')
    
    # Keep last row per group
    df_win = (
        df_win
        .groupby(['event_id', 'dg_id'], as_index=False)
        .last()
    )
    
    after_dedup = len(df_win)
    dropped = before_dedup - after_dedup
    
    print(f"\n   ✓ Deduplication complete:")
    print(f"     Before: {before_dedup:,} rows")
    print(f"     After:  {after_dedup:,} rows")
    print(f"     Dropped: {dropped:,} duplicate entries ({100*dropped/before_dedup:.2f}%)")

else:
    print(f"   ✓ No (event, player) duplicates found")

# FINAL VERIFICATION
print(f"\n{'='*70}")
print("✓ Deduplication complete")
print(f"{'='*70}")

final_dupes = df_win.duplicated(subset=['event_id', 'dg_id']).sum()
print(f"\nFinal duplicate check: {final_dupes} duplicates remaining")

assert final_dupes == 0, "ERROR: Duplicates still present after deduplication!"

print(f"✓ Final dataset shape: {df_win.shape[0]:,} rows × {df_win.shape[1]} columns")

STEP 9: REMOVE DUPLICATES

1. Checking for exact duplicate rows...
   Exact duplicate rows: 0
   ✓ No exact duplicates found

2. Checking for (event_id, player) duplicates...
   Duplicate (event_id, dg_id) pairs: 41

   ⚠️  WARNING: Same player appears multiple times in same event
   This is likely due to:
      - Multiple sportsbooks (DraftKings + FanDuel)
      - Multiple timestamps (opening + closing lines)
      - Data collection running at different times

   Example: Matsuyama, Hideki in 2025_2025-01-02
   This player appears multiple times with different odds:



,player_name,book,close_time,close_odds,dec_odds,Y
6,"Matsuyama, Hideki",draftkings,2025-01-02 11:38,2000,21.000000,1
65,"Matsuyama, Hideki",fanduel,2025-01-02 11:38,2000,21.000000,1



   DEDUPLICATION STRATEGY:
   Keep row with LATEST close_time per (event_id, dg_id)
   Rationale: Closing lines are most accurate, reflect latest info

   ✓ Deduplication complete:
     Before: 82 rows
     After:  41 rows
     Dropped: 41 duplicate entries (50.00%)

✓ Deduplication complete

Final duplicate check: 0 duplicates remaining
✓ Final dataset shape: 41 rows × 32 columns


In [22]:
# ============================================================================
# CELL 13: Final Validation Checks
# ============================================================================

print("="*70)
print("FINAL VALIDATION CHECKS")
print("="*70)

checks = {
    "1. No nulls in critical columns": 
        df_win[['event_id', 'dg_id', 'Y', 'dec_odds', 'p_book', 'implied_raw']].isna().sum().sum() == 0,
    
    "2. Unique (event, player) pairs": 
        df_win.duplicated(subset=['event_id', 'dg_id']).sum() == 0,
    
    "3. Decimal odds >= 1.0": 
        (df_win['dec_odds'] >= 1.0).all(),
    
    "4. Implied probs in (0, 1)": 
        ((df_win['implied_raw'] > 0) & (df_win['implied_raw'] < 1)).all(),
    
    "5. De-vigged probs in (0, 1)": 
        ((df_win['p_book'] > 0) & (df_win['p_book'] < 1)).all(),
    
    "6. De-vigged probs sum to 1.0 per event": 
        np.allclose(df_win.groupby('event_id')['p_book'].sum(), 1.0, atol=1e-10),
    
    "7. Overround >= 1.0 (all events)": 
        (df_win.groupby('event_id')['overround'].first() >= 1.0).all(),
    
    "8. Field size >= 2 (all events)": 
        (df_win['field_size'] >= 2).all(),
    
    "9. Price rank starts at 1": 
        (df_win.groupby('event_id')['price_rank'].min() == 1).all(),
    
    "10. Target variable is 0 or 1":
        df_win['Y'].isin([0, 1]).all() or df_win['Y'].isna().any()
}

all_passed = True
for check_name, passed in checks.items():
    status = "✓ PASS" if passed else "✗ FAIL"
    if not passed:
        all_passed = False
    print(f"{status}: {check_name}")

if not all_passed:
    print("\n⚠️  Some validation checks FAILED - review issues above")
else:
    print("\n✅ All validation checks PASSED")

FINAL VALIDATION CHECKS
✓ PASS: 1. No nulls in critical columns
✓ PASS: 2. Unique (event, player) pairs
✓ PASS: 3. Decimal odds >= 1.0
✓ PASS: 4. Implied probs in (0, 1)
✓ PASS: 5. De-vigged probs in (0, 1)
✗ FAIL: 6. De-vigged probs sum to 1.0 per event
✗ FAIL: 7. Overround >= 1.0 (all events)
✓ PASS: 8. Field size >= 2 (all events)
✗ FAIL: 9. Price rank starts at 1
✓ PASS: 10. Target variable is 0 or 1

⚠️  Some validation checks FAILED - review issues above


In [23]:
# ============================================================================
# CELL 14: Summary Statistics (for Sprint 2 Report)
# ============================================================================

print("="*70)
print("CLEANED DATASET SUMMARY (for Sprint 2 Report)")
print("="*70)

print(f"\n📊 Dataset Overview:")
print(f"  Total observations: {len(df_win):,}")
print(f"  Unique events: {df_win['event_id'].nunique():,}")
print(f"  Unique players: {df_win['dg_id'].nunique():,}")
print(f"  Date range: {df_win['close_time_dt'].min().date()} to {df_win['close_time_dt'].max().date()}")
print(f"  Years covered: {sorted(df_win['year_corrected'].unique())}")
print(f"  Sportsbooks: {df_win['book'].unique()}")

print(f"\n📊 Target Variable (Winners):")
total_winners = df_win['Y'].sum()
expected_winners = df_win['event_id'].nunique()
print(f"  Total winners: {int(total_winners)}")
print(f"  Expected winners (# events): {expected_winners}")
print(f"  Coverage: {100 * total_winners / expected_winners:.1f}%")
print(f"  Win rate: {df_win['Y'].mean():.6f} ({100*df_win['Y'].mean():.4f}%)")

print(f"\n📊 Odds Distribution:")
odds_summary = df_win[['dec_odds', 'implied_raw', 'p_book']].describe()
print(odds_summary)

print(f"\n📊 Event Characteristics:")
event_summary = df_win.groupby('event_id').agg({
    'field_size': 'first',
    'overround': 'first'
}).describe()
print(event_summary)

print(f"\n📊 Favorite Performance:")
top5_perf = df_win[df_win['is_top5_favorite']]
print(f"  Top-5 favorites win rate: {top5_perf['Y'].mean():.4f} ({100*top5_perf['Y'].mean():.2f}%)")
print(f"  Top-5 favorites total wins: {int(top5_perf['Y'].sum())} / {expected_winners} events")

rest_perf = df_win[~df_win['is_top5_favorite']]
print(f"  Non-favorites win rate: {rest_perf['Y'].mean():.4f} ({100*rest_perf['Y'].mean():.2f}%)")
print(f"  Non-favorites total wins: {int(rest_perf['Y'].sum())} / {expected_winners} events")

print(f"\n📊 Data Quality:")
events_1_winner = (df_win.groupby('event_id')['Y'].sum() == 1).sum()
print(f"  Events with exactly 1 winner: {events_1_winner} / {df_win['event_id'].nunique()}")
print(f"  Clean event coverage: {100 * events_1_winner / df_win['event_id'].nunique():.1f}%")

if events_1_winner < df_win['event_id'].nunique():
    problem_events = df_win['event_id'].nunique() - events_1_winner
    print(f"  ⚠️  {problem_events} events flagged for manual review")

CLEANED DATASET SUMMARY (for Sprint 2 Report)

📊 Dataset Overview:
  Total observations: 41
  Unique events: 37
  Unique players: 31
  Date range: 2025-01-02 to 2025-10-08
  Years covered: [np.int32(2025)]
  Sportsbooks: ['fanduel' 'draftkings']

📊 Target Variable (Winners):
  Total winners: 41
  Expected winners (# events): 37
  Coverage: 110.8%
  Win rate: 1.000000 (100.0000%)

📊 Odds Distribution:
        dec_odds  implied_raw    p_book
count  41.000000    41.000000 41.000000
mean   61.878049     0.062438  0.447813
std    72.099111     0.087525  0.108018
min     3.100000     0.002494  0.096491
25%    15.000000     0.012346  0.470930
50%    41.000000     0.024390  0.500000
75%    81.000000     0.066667  0.500000
max   401.000000     0.322581  0.544910

📊 Event Characteristics:
       field_size  overround
count   37.000000  37.000000
mean     2.216216   0.138145
std      0.629600   0.184567
min      2.000000   0.005816
25%      2.000000   0.024147
50%      2.000000   0.048780
75%    

In [24]:
print("WINNER IDENTIFICATION DIAGNOSTIC")
print("="*70)

print("\nbet_outcome_numeric distribution:")
print(df_raw['bet_outcome_numeric'].value_counts())

print("\noutcome distribution:")
print(df_raw['outcome'].value_counts())

print("\nbet_outcome_text distribution:")
print(df_raw['bet_outcome_text'].value_counts())

print("\nSample of winners vs losers:")
print(df_raw[['player_name', 'outcome', 'bet_outcome_numeric', 'bet_outcome_text']].head(10))

WINNER IDENTIFICATION DIAGNOSTIC

bet_outcome_numeric distribution:
bet_outcome_numeric
0    9915
1      82
Name: count, dtype: int64

outcome distribution:
outcome
CUT    4290
T34     151
T18     112
T44     110
WD      108
       ... 
42        2
44        2
46        2
47        2
30        2
Name: count, Length: 141, dtype: int64

bet_outcome_text distribution:
bet_outcome_text
loss            9915
paid in full      82
Name: count, dtype: int64

Sample of winners vs losers:
          player_name outcome  bet_outcome_numeric bet_outcome_text
0  Schauffele, Xander     T30                    0             loss
1      Thomas, Justin     T26                    0             loss
2    Morikawa, Collin       2                    0             loss
3       Aberg, Ludvig      T5                    0             loss
4    Cantlay, Patrick     T15                    0             loss
5         Im, Sungjae       3                    0             loss
6   Matsuyama, Hideki       1            